In [ ]:
import random, glob
import numpy as np

In [ ]:
music_files = [a for a in glob.glob("dataset/*/*")]
print("A random song", random.sample(music_files, 1))

In [ ]:
from music21 import midi
def play_midi_file(midi_file_name):
    mf = midi.MidiFile()

    mf.open(midi_file_name) # path='abc.midi'
    mf.read()
    mf.close()
    s = midi.translate.midiFileToStream(mf)
    s.show('midi')

In [ ]:
len(music_files)

In [ ]:
from music21 import converter,corpus, chord

In [ ]:
tonic_mode_all = []
for i, name in enumerate(music_files[:5]):
    try:
        parsed_score = converter.parse(name)
        key_of_one = parsed_score.analyze('key')
        tonic_mode_all.append((i, name, key_of_one.tonic.name, key_of_one.mode))
    except:
        print(f'Error at {i}')
        tonic_mode_all.append((i, name, 'NAN', 'NAN'))
        continue
    print(f'{tonic_mode_all[i]}')

In [ ]:
def get_chords(music_files):
    chords = []
    for file_no, filename in enumerate(music_files):
        try:
            chords.append(converter.parse(filename).chordify())
            print(f'Happening of {file_no}', "filename = ", filename)
            
        except:
            print(f'Happening of {file_no}', "filename = ", filename)
            print("file failed!!!!!")
            continue
        
    
    return chords

In [ ]:
music_files_25 = music_files[:25]
#get_chords(music_files[:10])

In [ ]:
chords_of_all_music = get_chords(music_files)

In [ ]:
#compressed chords if all music
chords_of_all_music[:5]

In [ ]:
from music21 import chord, duration

Classifying the music into different modes
- firstly, let's code for major mode

In [ ]:
from music21 import *

In [ ]:
# for seeing output from chords and given duration:
def get_music_midi_from_chords_and_duration(input_chords):
    midi_stream = stream.Stream()

    for note_pattern, duration_pattern in input_chords:
        notes_in_chord = note_pattern.split('.')
        
        chord_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(current_note)
            new_note.duration = duration.Duration(duration_pattern)
            new_note.storedInstrument = instrument.Violoncello()
            chord_notes.append(new_note)
        new_chord = chord.Chord(chord_notes)
        
        midi_stream.append(new_chord)

        new_tempo = tempo.MetronomeMark(number=50)
            
        midi_stream.append(new_tempo)

    midi_stream = midi_stream.chordify()
    timestr = time.strftime("%Y%m%d-%H%M%S")
    new_file = 'output-' + timestr + '.mid'
    return midi_stream.write('midi', fp=new_file)

In [ ]:
def round_chord_durations(number):
    if(number>1.25):
        return 2
    if(number>.30):
        return 1.25
    if(number>.10):
        return 0.3
    return 0.1

For chord and duration of a single song(music file)

In [ ]:
def get_chord_and_duration_data(indv_midi_chordified):
    chord_duration_data = []
    for element in indv_midi_chordified.flat:
        if isinstance(element, chord.Chord):
            chord_duration = round_chord_durations(element.duration.quarterLength)
            chord_name = ".".join([n.nameWithOctave for n in element.pitches])
            chord_duration_data.append((chord_name, chord_duration))

    return chord_duration_data

In [ ]:
chords_and_duration_data_all_music= []
for one_music_chords in chords_of_all_music:
   chords_and_duration_data_all_music.append((get_chord_and_duration_data(one_music_chords)))

In [ ]:
# this is the main data:
type(chords_and_duration_data_all_music[0][0][1])

In [ ]:
print("Generating music from our processed chords...")
proccessed_chords_to_midi_sample = get_music_midi_filename_from_chords(chords_and_duration_data_all_music[0][4:50])
print(proccessed_chords_to_midi_sample)
play_midi_file(proccessed_chords_to_midi_sample)

from Tonic_mode_all, separating major and minor songs 

In [ ]:
tonic_mode_all[0][0], tonic_mode_all[0][-1]

In [ ]:
all_major_songs_chords_and_duration = []
all_minor_songs_chords_and_duration = []
for i in range(len(tonic_mode_all)):
    if(tonic_mode_all[i][-1] == 'major'):
        try:
            all_major_songs_chords_and_duration.append(chords_and_duration_data_all_music[i])
        except:
            all_major_songs_chords_and_duration.append('NANNNNNN')
    else:
        try:
            all_minor_songs_chords_and_duration.append(chords_and_duration_data_all_music[i])
        except:
             all_minor_songs_chords_and_duration.append('NANNNNNN')

In [ ]:
len(all_minor_songs_chords_and_duration), #minor_songs_chords_with_duration, len(minor_songs_chords_with_duration)

In [ ]:
major_dataset = all_major_songs_chords_and_duration 
minor_dataset = all_minor_songs_chords_and_duration

In [ ]:
#just a reference code
#major_dataset
#y=np.array(major_dataset)
#unique = set(major_dataset_all)
#unique2 = set(minor_dataset_all)

In [ ]:
chords_and_duration_data_all_music

In [ ]:
main_dataset = chords_and_duration_data_all_music

* Data preparation stage

In [ ]:
no_of_timesteps = 32
x = []
y = []
## CD stands for chord and duration.

for each_music_with_CD in main_dataset:
    for each_CD in range(0, len(each_music_with_CD) - no_of_timesteps,  1):
        
        ## preparing input and output sequences:
        input_ = each_music_with_CD[i:i + no_of_timesteps]
        output = each_music_with_CD[i + no_of_timesteps]
        
        for i in range(len(input_)):
            input_[i]= "@".join(map(str,input_[i]))
            
        output = "@".join(map(str,output))
        
        x.append(input_)
        y.append(output)
        
x=np.array(x)
y=np.array(y)

In [ ]:
y

In [ ]:
#assigning unique integer to every chords_and_duration

unique_x_CD = list(set(x.ravel()))
unique_x_CD_to_int = dict((chord_and_duration, number) for number, chord_and_duration in enumerate(unique_x_CD))
unique_x_CD_to_int

In [ ]:
#preparing input sequences::

x_seq=[]
for each_row in x:
    temp=[]
    for each_piece in each_row:
        #assigning unique integer to every note
        temp.append(unique_x_CD_to_int[each_piece])
    x_seq.append(temp)
    
x_seq = np.array(x_seq)
x_seq.shape

In [ ]:
# preparing th output sequences as well::

unique_y_CD = list(set(y))
unique_y_CD_to_int = dict((chord_and_duration, number) for number, chord_and_duration in enumerate(unique_y_CD)) 
unique_y_CD_to_int

In [ ]:
y_seq=np.array([unique_y_CD_to_int[i] for i in y])
y_seq.shape

In [ ]:
# preserving 80% of the data for training and the rest 20% for the evaluation:

from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_seq,y_seq,test_size=0.2,random_state=0)

In [ ]:
len(x_val)

# model building phase:

In [ ]:
tf.config.experimental.list_physical_devices()

In [ ]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *
import keras.backend as K

K.clear_session()
model = Sequential()
    
#embedding layer
model.add(Embedding(len(unique_x_CD), 100, input_length=32,trainable=True)) 

model.add(Conv1D(64,3, padding='causal',activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))
    
model.add(Conv1D(128,3,activation='relu',dilation_rate=2,padding='causal'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))

model.add(Conv1D(256,3,activation='relu',dilation_rate=4,padding='causal'))
model.add(Dropout(0.2))
model.add(MaxPool1D(2))
          
#model.add(Conv1D(256,5,activation='relu'))    
model.add(GlobalMaxPool1D())
    
model.add(Dense(256, activation='relu'))
model.add(Dense(len(unique_y_CD), activation='softmax'))
    
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

model.summary()

In [ ]:
def lstm():
    K.clear_session()
    model = Sequential()
    #embedding layer
    model.add(Embedding(len(unique_x_CD), 100, input_length=32,trainable=True)) 
    model.add(LSTM(128,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128))
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(Activation('relu'))

    model.add(Dense(len(unique_y_CD), activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
    return model

In [ ]:
model = lstm()
model.summary()

In [ ]:
!pip install h5py

In [ ]:
import h5py

In [ ]:
#defining call back to save the best model during training>
mc=ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True,verbose=1)

In [ ]:
#actual training
history = model.fit(np.array(x_train),np.array(y_train),batch_size=128,epochs=10, 
                   validation_data=(np.array(x_val),np.array(y_val)),verbose=1, callbacks=[mc])

In [ ]:
#loading best model (Previously trained modle)
from keras.models import load_model
model = load_model('best_model.h5')


In [ ]:
import numpy as np
import random
ind = np.random.randint(0,len(x_val)-1)
random_music = x_val[ind]
random_music

In [ ]:
predictions=[]
for i in range(10):

    random_music = random_music.reshape(1,no_of_timesteps)
    print("random music = ", random_music)
    

    prob  = model.predict(random_music)[0]
    y_pred= np.argmax(prob,axis=0)
    predictions.append(y_pred)

    random_music = np.insert(random_music[0],len(random_music[0]),y_pred)
    random_music = random_music[1:]
    
print(predictions)

In [ ]:
#intergers back to notes
unique_x_int_to_CD = dict((number, note_) for number, note_ in enumerate(unique_x_CD)) 
unique_x_int_to_CD

In [ ]:
predicted_CD = [unique_x_int_to_CD[i] for i in predictions]
predicted_CD

In [ ]:
predicted_CD_split = []
for each_outcome in predicted_CD:
    temp_list = []
    temp_list = each_outcome.split("@")
    temp_list[1] =float(temp_list[1])
    predicted_CD_split.append(tuple(temp_list))
    

predicted_CD_split

In [ ]:
print("Generating music from our processed chords...")
proccessed_chords_to_midi_sample = get_music_midi_from_chords_and_duration(predicted_CD_split)
print(proccessed_chords_to_midi_sample)
play_midi_file(proccessed_chords_to_midi_sample)